# 1. Scrape Wikipedia for List of Mexico City's Metro stations

In [1]:
#import required libraries for scraping
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_Mexico_City_Metro_stations"  
page = urllib.request.urlopen(url)  #Open url and put HTML code into the page variable

In [3]:
soup = BeautifulSoup(page, 'lxml') #Parse HTML 

In [4]:
all_tables = soup.find_all('table')  #retrieve all table classes 
#all_tables  

In [5]:
metro_stations_table = soup.find('table',class_='wikitable sortable plainrowheaders') #table containing stations  
#metro_stations_table

In [6]:
#Extract station name, line numer, year-opened, type of station, encoded URL from 'metro_stations_table'
station = []
line = []
opened = []
grade = []
encoded_url = []

for row in metro_stations_table.find_all('tr')[1:]:
    cells = row.find_all('td')
    
    #Extract Station Name
    a_tag = cells[0].find('a')
    station.append(a_tag.find(text=True))
    
    #Extract Line Number
    a_tag2 = cells[1].find_all('a')
    if len(a_tag2) == 1:
        line.append(a_tag2[0].attrs['title'])
    else:
        temp = []
        for elements in a_tag2:
            temp.append(elements.attrs['title'])
        test = ','.join(temp)
        line.append(test)
    
    #Extract Year the station opened.
    opened.append(cells[2].find(text=True))
    
    #Extract the type of station. 
    grade.append(cells[3].find(text=True))
    
    #Extract Encoded URL
    encoded_url.append(cells[0].find('a').attrs['href'])

In [7]:
#Uncommnet to view format of cells variable
#cells

In [8]:
#Create dataframe for stations data
df_stations = pd.DataFrame({'Station':station,'Line':line,'Year-Built':opened,'Grade':grade, 'URL':encoded_url})
df_stations

,Station,Line,Year-Built,Grade,URL
0,Acatitla,Mexico City Metro Line A,1991\n,At-grade\n,/wiki/Acatitla_metro_station
1,Aculco,Mexico City Metro Line 8,1994\n,At-grade\n,/wiki/Aculco_metro_station
2,Agrícola Oriental,Mexico City Metro Line A,1991\n,At-grade\n,/wiki/Agr%C3%ADcola_Oriental_metro_station
3,Allende,Mexico City Metro Line 2,1970\n,Underground\n,/wiki/Allende_metro_station
4,Apatlaco,Mexico City Metro Line 8,1994\n,At-grade\n,/wiki/Apatlaco_metro_station
...,...,...,...,...,...
158,Xola,Mexico City Metro Line 2,1970\n,At-grade\n,/wiki/Xola_metro_station
159,Zapata,"Mexico City Metro Line 3,Mexico City Metro Lin...",1980\n,Underground\n,/wiki/Zapata_metro_station
160,Zapotitlán,Mexico City Metro Line 12,2012\n,Elevated\n,/wiki/Zapotitl%C3%A1n_metro_station
161,Zaragoza,Mexico City Metro Line 1,1969\n,Underground\n,/wiki/Zaragoza_metro_station_(Mexico_City)


In [9]:
df_stations['Line'] = df_stations['Line'].replace(to_replace = 'Mexico City Metro Line', value = '', regex=True)
df_stations['Year-Built'] = df_stations['Year-Built'].replace(to_replace = '\n', value = '', regex=True)
df_stations['Grade'] = df_stations['Grade'].replace(to_replace = '\n', value = '', regex=True)

In [10]:
url = 'https://en.wikipedia.org' + df_stations['URL']

In [11]:
#Loop to obtain geospatial coordinates for each station
lat = []
lon = []

for i in url:
    page = urllib.request.urlopen(i)
    soup = BeautifulSoup(page, 'lxml')
    infobox_table = soup.find('table',class_ ='infobox vcard')  #Obtain infobox table
    coordinates_raw = infobox_table.find("span", class_= 'geo').find(text=True)  #Obtain coordinates 
    test = coordinates_raw.split(';')
    
    #Append Data
    lat.append(float(test[0]))
    lon.append(float(test[1]))

In [12]:
#Uncomment to view format of 'infobox_table' (To view where coordinate data is located within variable)
#infobox_table

In [13]:
#Append coordinates data to df_stations dataframe
df_stations['Latitude'] = lat
df_stations['Longitude'] = lon

In [14]:
df_stations.head()

,Station,Line,Year-Built,Grade,URL,Latitude,Longitude
0,Acatitla,A,1991,At-grade,/wiki/Acatitla_metro_station,19.364737,-99.005892
1,Aculco,8,1994,At-grade,/wiki/Aculco_metro_station,19.374069,-99.108095
2,Agrícola Oriental,A,1991,At-grade,/wiki/Agr%C3%ADcola_Oriental_metro_station,19.404835,-99.069901
3,Allende,2,1970,Underground,/wiki/Allende_metro_station,19.435514,-99.137406
4,Apatlaco,8,1994,At-grade,/wiki/Apatlaco_metro_station,19.379292,-99.109597


In [15]:
df_stations = df_stations.drop(columns = 'URL')

# 2. Visualize Metro Stations around Mexico City 

In [16]:
#import required libraries
from geopy.geocoders import Nominatim #Used to convert an address into geographic coordinates
import folium #map rendering library

In [17]:
#Obtain coordinates for Mexico City

geolocator = Nominatim(user_agent='mexico_city')  #Define geolocator instance
location = geolocator.geocode('Mexico City, MX')
latitude = location.latitude
longitude = location.longitude


print(location,':',latitude,',',longitude)

Ciudad de México, México : 19.4326296 , -99.1331785


In [18]:
#Map metro stations

map_cdmx = folium.Map(location=[latitude, longitude], zoom_start = 11)  #Create Map
for lat, lng, label in zip(df_stations['Latitude'], df_stations['Longitude'], df_stations['Station']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], radius = 5, popup = label, fill = True, fill_opacity = 0.7).add_to(map_cdmx)

map_cdmx

In [19]:
import geopy.distance

__Determine Closest Station from each Station__

In [20]:
closest_station = []
dist_to_closest_station = []

for row in df_stations.itertuples():
    coords_1 = (row[5],row[6])
    
    geodesic_distance = []  #STORE DIST. BETWEEN STATION AND EVERY OTHER STATION, NOTE TEMP STORAGE
    
    for line in df_stations.itertuples():
        coords_2 = (line[5],line[6])
        
        dist = geopy.distance.distance(coords_1, coords_2).m  #geodesic distance between stations
        geodesic_distance.append(dist)
        
    min_dist = min(i for i in geodesic_distance if i > 0)
    position = geodesic_distance.index(min_dist)
    dist_to_closest_station.append(round(min_dist))
    
    closest_station.append(df_stations.loc[position, 'Station'])
    

In [21]:
df_stations['Closest Station'] = closest_station
df_stations['Distance to Closest Station'] = dist_to_closest_station

In [22]:
df_stations.head()

,Station,Line,Year-Built,Grade,Latitude,Longitude,Closest Station,Distance to Closest Station
0,Acatitla,A,1991,At-grade,19.364737,-99.005892,Santa Marta,1272
1,Aculco,8,1994,At-grade,19.374069,-99.108095,Apatlaco,599
2,Agrícola Oriental,A,1991,At-grade,19.404835,-99.069901,Pantitlán,1188
3,Allende,2,1970,Underground,19.435514,-99.137406,Bellas Artes,484
4,Apatlaco,8,1994,At-grade,19.379292,-99.109597,Aculco,599


In [23]:
#CLASS FOR BOLD OUTPUT
class color:
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [24]:
#SOME METRICS ABOUT STATIONS: 

#MEAN 'CLOSEST STATION' DISTANCE.  
mean = df_stations['Distance to Closest Station'].mean()

#PAIR OF 'CLOSEST STATIONS' THAT ARE THE FURTHEST APART.  
max_ = df_stations['Distance to Closest Station'].max()
max_station = df_stations[df_stations['Distance to Closest Station'] == max_].iloc[0]['Station']
max_closest_station = df_stations[df_stations['Distance to Closest Station'] == max_].iloc[0]['Closest Station']

#PAIR OF 'CLOSEST STATIONS' THAT ARE THE CLOSEST TO EACH OTHER. 
min_ = df_stations['Distance to Closest Station'].min()
min_station = df_stations[df_stations['Distance to Closest Station'] == min_].iloc[0]['Station']
min_closest_station = df_stations[df_stations['Distance to Closest Station'] == min_].iloc[0]['Closest Station']



print('Mean Distance between Stations:', color.BOLD + str(round(mean)) + color.END, 'm.')

print(color.BOLD + max_station + color.END, 'and', color.BOLD + max_closest_station + color.END, 
      'are the two adjacent stations furthest apart at', color.BOLD + str(max_) + color.END, 'm.')

print(color.BOLD + min_station + color.END, 'and', color.BOLD + min_closest_station + color.END,
      'are the two stations closest to each other at', color.BOLD + str(min_) + color.END, 'm.')

Mean Distance between Stations: 901 m.
La Paz and Los Reyes are the two adjacent stations furthest apart at 1938 m.
Colegio Militar and Popotla are the two stations closest to each other at 402 m.


In [25]:
df_stations[df_stations['Station'] == 'Acatitla']

,Station,Line,Year-Built,Grade,Latitude,Longitude,Closest Station,Distance to Closest Station
0,Acatitla,A,1991,At-grade,19.364737,-99.005892,Santa Marta,1272


# 3. Foursquare API

In [26]:
import json
import requests

In [27]:
#FOURSQURE API
CLIENT_ID = 'IXD2VZKCPKWMYM2MVCIDROKTLRSI4HKHKSTV2S35WTPLLFA0' 
CLIENT_SECRET = '1LPGU40JIT4FOQEQJQF2HLM1MHEVEYRBR1HXOKUJSO5IKXGD' 
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: IXD2VZKCPKWMYM2MVCIDROKTLRSI4HKHKSTV2S35WTPLLFA0
CLIENT_SECRET:1LPGU40JIT4FOQEQJQF2HLM1MHEVEYRBR1HXOKUJSO5IKXGD


In [28]:
#GET request URL
radius = 300
lat = df_stations.loc[0, 'Latitude']
lng = df_stations.loc[0, 'Longitude']

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    lng, 
    radius, 
    LIMIT)

print('Request for', '\033[1m' + df_stations.loc[0, 'Station'] + '\033[0m', 'Station', '(', lat, lng, ')')
print(url)

Request for Acatitla Station ( 19.364737 -99.005892 )
https://api.foursquare.com/v2/venues/explore?&client_id=IXD2VZKCPKWMYM2MVCIDROKTLRSI4HKHKSTV2S35WTPLLFA0&client_secret=1LPGU40JIT4FOQEQJQF2HLM1MHEVEYRBR1HXOKUJSO5IKXGD&v=20180605&ll=19.364737,-99.005892&radius=300&limit=100


In [29]:
#Send GET request
results = requests.get(url).json()
#Uncommnet to view how data is structured
#results

In [30]:
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '5ab96a96e1f228669fc9ea99',
   'name': 'La Esperanza',
   'location': {'address': 'Ignacio Zaragoza',
    'lat': 19.363365207453622,
    'lng': -99.00472196618074,
    'labeledLatLngs': [{'label': 'display',
      'lat': 19.363365207453622,
      'lng': -99.00472196618074}],
    'distance': 196,
    'cc': 'MX',
    'city': 'Ciudad de México',
    'state': 'Distrito Federal',
    'country': 'México',
    'formattedAddress': ['Ignacio Zaragoza',
     'Ciudad de México, Distrito Federal',
     'México']},
   'categories': [{'id': '4bf58dd8d48988d16a941735',
     'name': 'Bakery',
     'pluralName': 'Bakeries',
     'shortName': 'Bakery',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-

In [31]:
nearby_venues = pd.json_normalize(venues)  #flatten JSON
nearby_venues.head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.crossStreet,venue.location.postalCode
0,e-0-5ab96a96e1f228669fc9ea99-0,0,"[{'summary': 'This spot is popular', 'type': '...",5ab96a96e1f228669fc9ea99,La Esperanza,Ignacio Zaragoza,19.363365,-99.004722,"[{'label': 'display', 'lat': 19.36336520745362...",196,MX,Ciudad de México,Distrito Federal,México,"[Ignacio Zaragoza, Ciudad de México, Distrito ...","[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",0,[],NaN,NaN
1,e-0-56807381498eabd7d85a388f-1,0,"[{'summary': 'This spot is popular', 'type': '...",56807381498eabd7d85a388f,little caesar,Av. Zaragoza,19.364720,-99.006484,"[{'label': 'display', 'lat': 19.36472042863899...",62,MX,Oasis Acatitla,México,México,"[Av. Zaragoza, Oasis Acatitla, México, México]","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],NaN,NaN
2,e-0-4d138c26d1848cfaede3c471-2,0,"[{'summary': 'This spot is popular', 'type': '...",4d138c26d1848cfaede3c471,Toks,Calz. Ignacio Zaragoza 1886,19.366074,-99.005900,"[{'label': 'display', 'lat': 19.36607422482322...",148,MX,Iztapalapa,Distrito Federal,México,[Calz. Ignacio Zaragoza 1886 (Esq. con Octavio...,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",0,[],Esq. con Octavio Paz,09510
3,e-0-51fd69e5498ec316901c73db-3,0,"[{'summary': 'This spot is popular', 'type': '...",51fd69e5498ec316901c73db,Tender's Coffe,NaN,19.366628,-99.004765,"[{'label': 'display', 'lat': 19.366628, 'lng':...",241,MX,NaN,NaN,México,[México],"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",0,[],NaN,NaN
4,e-0-513cc414e4b00e248b4a7123-4,0,"[{'summary': 'This spot is popular', 'type': '...",513cc414e4b00e248b4a7123,Domino's Pizza,"Calzada Gral Ignacio Zaragoza No. 3254, Local ...",19.366939,-99.005129,"[{'label': 'display', 'lat': 19.36693883742614...",257,MX,Iztapalapa,Distrito Federal,México,"[Calzada Gral Ignacio Zaragoza No. 3254, Local...","[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",0,[],Prolongación Octavio Paz,09150


In [32]:
nearby_venues.shape

(11, 20)

In [33]:
#Retreive only useful columns
venue_id = nearby_venues['venue.id']
venue_name = nearby_venues['venue.name']
venue_categories = nearby_venues['venue.categories']
latitude_column = nearby_venues['venue.location.lat']
longitude_column = nearby_venues['venue.location.lng']
distance = nearby_venues['venue.location.distance']

nearby_venues = pd.DataFrame({'Venue Id':venue_id,
                     'Name':venue_name,
                     'Category':venue_categories,
                     'Latitude':latitude_column,
                     'Longitude':longitude_column,
                     'Distance (m)':distance})
nearby_venues.head()

#Note: venue.categories column needs to be cleaned and formatted (Done futher down)

,Venue Id,Name,Category,Latitude,Longitude,Distance (m)
0,5ab96a96e1f228669fc9ea99,La Esperanza,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",19.363365,-99.004722,196
1,56807381498eabd7d85a388f,little caesar,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",19.364720,-99.006484,62
2,4d138c26d1848cfaede3c471,Toks,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",19.366074,-99.005900,148
3,51fd69e5498ec316901c73db,Tender's Coffe,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",19.366628,-99.004765,241
4,513cc414e4b00e248b4a7123,Domino's Pizza,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",19.366939,-99.005129,257


__Repeat process for all Metro Stations:__

In [34]:
#Create empty df to store venue information (venue id, name, category, station, coordinates, etc)
df_venues = pd.DataFrame()


for row in df_stations.itertuples():
    station = row[1]
    lat = row[5]
    lng = row[6]

    radius  = 450 #1/2 mean calculated above
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        radius, 
        LIMIT)
    
    #Send GET request
    results = requests.get(url).json()
        
        
    venues = results['response']['groups'][0]['items']  #Venues data, JSON format. 
    df_temp = pd.json_normalize(venues) #Normalize JSON data into a flat table.
        
        
    #Format data into usable dataframe (df_venues)
    if df_temp.empty:
        pass
    else:
        #Retrieve useful columns and merge onto dataframe. 
        venue_id = df_temp['venue.id']
        venue_name = df_temp['venue.name']
        venue_categories = df_temp['venue.categories']
        latitude_column = df_temp['venue.location.lat']
        longitude_column = df_temp['venue.location.lng']
        distance = df_temp['venue.location.distance']

        df_short = pd.DataFrame({'Venue Id':venue_id,
                             'Name':venue_name,
                             'Category':venue_categories,
                             'Latitude':latitude_column,
                             'Longitude':longitude_column,
                             'Distance (m)':distance})
        
        
        station_name = [station] * len(df_short.index) #List containing station repeatedly (['Zocalo, ..., 'Zocalo])
        df_short.insert(2, 'Station', station_name)  #Insert list as a column. 
                              
        df_venues = df_venues.append(df_short, ignore_index=True) #Append df created for each category. 
     
    print(station, '(', lat,',',lng,')',':', results['response']['totalResults'])
    


Acatitla ( 19.364737 , -99.005892 ) : 12
Aculco ( 19.374069 , -99.108095 ) : 16
Agrícola Oriental ( 19.404835 , -99.069901 ) : 10
Allende ( 19.435514 , -99.137406 ) : 115
Apatlaco ( 19.379292 , -99.109597 ) : 12
Aquiles Serdán ( 19.490423 , -99.19487 ) : 26
Aragón ( 19.451236 , -99.095886 ) : 8
Atlalilco ( 19.356134 , -99.101315 ) : 13
Auditorio ( 19.425498 , -99.191995 ) : 42
Autobuses del Norte ( 19.478973 , -99.140668 ) : 26
Balbuena ( 19.423231 , -99.102302 ) : 20
Balderas ( 19.42744 , -99.149036 ) : 14
Barranca del Muerto ( 19.360648 , -99.190149 ) : 69
Bellas Artes ( 19.436243 , -99.141955 ) : 37
Bondojito ( 19.46461 , -99.111958 ) : 21
Bosque de Aragón ( 19.458115 , -99.069192 ) : 16
Boulevard Puerto Aéreo ( 19.41967 , -99.09595 ) : 12
Buenavista ( 19.446603 , -99.153199 ) : 63
Calle 11 ( 19.320463 , -99.085734 ) : 24
Camarones ( 19.479135 , -99.190063 ) : 30
Canal del Norte ( 19.449112 , -99.116507 ) : 19
Canal de San Juan ( 19.398683 , -99.059365 ) : 20
Candelaria ( 19.428837 

In [35]:
df_venues

,Venue Id,Name,Station,Category,Latitude,Longitude,Distance (m)
0,5ab96a96e1f228669fc9ea99,La Esperanza,Acatitla,"[{'id': '4bf58dd8d48988d16a941735', 'name': 'B...",19.363365,-99.004722,196
1,56807381498eabd7d85a388f,little caesar,Acatitla,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",19.364720,-99.006484,62
2,4d138c26d1848cfaede3c471,Toks,Acatitla,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",19.366074,-99.005900,148
3,51fd69e5498ec316901c73db,Tender's Coffe,Acatitla,"[{'id': '4bf58dd8d48988d147941735', 'name': 'D...",19.366628,-99.004765,241
4,513cc414e4b00e248b4a7123,Domino's Pizza,Acatitla,"[{'id': '4bf58dd8d48988d1ca941735', 'name': 'P...",19.366939,-99.005129,257
...,...,...,...,...,...,...,...
5262,4f8ee79be4b0bdaf06d915c4,Café del Centro,Zócalo,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",19.435825,-99.132837,286
5263,4ddd467fd164236dd386de19,Terraza La Deli,Zócalo,"[{'id': '4bf58dd8d48988d110941735', 'name': 'I...",19.435857,-99.131562,322
5264,50115aeaf2e787781f5f0876,Café Ciudad de México,Zócalo,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",19.432141,-99.134446,203
5265,4c82bc1347cc224b1d14899f,MineraliA Outlet,Zócalo,"[{'id': '4bf58dd8d48988d127951735', 'name': 'A...",19.430370,-99.135395,413


In [36]:
#function that extracts venue's category
def get_category_type(row):
    categories_list = row['Category']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [37]:
df_venues['Category'] = df_venues.apply(get_category_type, axis=1)

df_venues

,Venue Id,Name,Station,Category,Latitude,Longitude,Distance (m)
0,5ab96a96e1f228669fc9ea99,La Esperanza,Acatitla,Bakery,19.363365,-99.004722,196
1,56807381498eabd7d85a388f,little caesar,Acatitla,Pizza Place,19.364720,-99.006484,62
2,4d138c26d1848cfaede3c471,Toks,Acatitla,Mexican Restaurant,19.366074,-99.005900,148
3,51fd69e5498ec316901c73db,Tender's Coffe,Acatitla,Diner,19.366628,-99.004765,241
4,513cc414e4b00e248b4a7123,Domino's Pizza,Acatitla,Pizza Place,19.366939,-99.005129,257
...,...,...,...,...,...,...,...
5262,4f8ee79be4b0bdaf06d915c4,Café del Centro,Zócalo,Café,19.435825,-99.132837,286
5263,4ddd467fd164236dd386de19,Terraza La Deli,Zócalo,Italian Restaurant,19.435857,-99.131562,322
5264,50115aeaf2e787781f5f0876,Café Ciudad de México,Zócalo,Café,19.432141,-99.134446,203
5265,4c82bc1347cc224b1d14899f,MineraliA Outlet,Zócalo,Arts & Crafts Store,19.430370,-99.135395,413


# 4. Exploratory Data Analysis

In [38]:
list_of_categories = sorted(df_venues.Category.unique())
print(len(list_of_categories))
list_of_categories

334


['ATM',
 'Accessories Store',
 'Advertising Agency',
 'African Restaurant',
 'Airport',
 'Airport Gate',
 'Airport Lounge',
 'Airport Service',
 'American Restaurant',
 'Antique Shop',
 'Arcade',
 'Arepa Restaurant',
 'Argentinian Restaurant',
 'Art Gallery',
 'Art Museum',
 'Arts & Crafts Store',
 'Arts & Entertainment',
 'Asian Restaurant',
 'Athletics & Sports',
 'Auto Dealership',
 'Auto Garage',
 'Automotive Shop',
 'BBQ Joint',
 'Baby Store',
 'Bagel Shop',
 'Bakery',
 'Bank',
 'Bar',
 'Baseball Field',
 'Baseball Stadium',
 'Basketball Court',
 'Basketball Stadium',
 'Bed & Breakfast',
 'Beer Bar',
 'Beer Garden',
 'Beer Store',
 'Big Box Store',
 'Bike Rental / Bike Share',
 'Bistro',
 'Bookstore',
 'Botanical Garden',
 'Boutique',
 'Boxing Gym',
 'Brazilian Restaurant',
 'Breakfast Spot',
 'Brewery',
 'Bridal Shop',
 'Bubble Tea Shop',
 'Buffet',
 'Building',
 'Burger Joint',
 'Burrito Place',
 'Bus Line',
 'Bus Station',
 'Bus Stop',
 'Business Service',
 'Butcher',
 'Cafeter

In [39]:
#IDENTIFY TEN MOST COMMON VENUE CATEGORIES
most_common_categories = df_venues.groupby('Category').size().sort_values(ascending=False).to_frame().reset_index()
most_common_categories.columns = ['Category','No. of Venues']  #Rename Columns
most_common_categories[0:10]

,Category,No. of Venues
0,Mexican Restaurant,571
1,Taco Place,395
2,Coffee Shop,213
3,Bakery,171
4,Restaurant,163
5,Ice Cream Shop,132
6,Pizza Place,118
7,Café,105
8,Bar,104
9,Convenience Store,102


In [40]:
#STATIONS WITH THE LEAST NUMBER OF VENUES SURROUNDING THEM. 
venues_per_station = df_venues.groupby('Station').size().sort_values().to_frame().reset_index()
venues_per_station.columns = ['Station', 'No. of Venues']

low_venue_stations = venues_per_station.loc[venues_per_station['No. of Venues'] < 5]

low_venue_stations

,Station,No. of Venues
0,La Paz,4
1,Martín Carrera,4


In [41]:
#Get all hotels & hostels nearby stations. 
hotels = df_venues.loc[(df_venues['Category'] == 'Hotel') | (df_venues['Category'] == 'Hostel')]

#Drop duplicates, keep record with the shortest distance from station
hotels = hotels.sort_values('Distance (m)')  #Sort by distance from station. 
hotels = hotels.drop_duplicates(subset='Venue Id', keep='first') #keep first occurrence.

#One-hot Encoding
hotel_dummies = pd.get_dummies(hotels['Category'])
hotels.insert(7,'One-hot Encoding', hotel_dummies['Hotel'])


hotels = hotels.sort_values('Station').reset_index(drop=True)
hotels[0:15]

,Venue Id,Name,Station,Category,Latitude,Longitude,Distance (m),One-hot Encoding
0,4eb83dfb4901cef9d9b877ba,Downtown México,Allende,Hotel,19.432875,-99.136208,319,1
1,55f476fe498e26410387a491,Hotel Historico Central,Allende,Hotel,19.433153,-99.138376,281,1
2,4b607499f964a520fae629e3,Hyatt Regency Mexico City,Auditorio,Hotel,19.427121,-99.192676,194,1
3,4b0586fdf964a520797922e3,Presidente InterContinental,Auditorio,Hotel,19.427262,-99.193754,269,1
4,4b0586fdf964a520b57922e3,W Hotel,Auditorio,Hotel,19.427293,-99.195377,407,1
5,4d008ec11ebe6dcb910a8891,Hotel Brasilia,Autobuses del Norte,Hotel,19.475595,-99.138973,415,1
6,4ef1990249010b41f48f641d,Hotel La Selva,Chabacano,Hotel,19.408195,-99.135025,81,1
7,4b0586fcf964a5206c7922e3,Four Seasons Hotel,Chapultepec,Hotel,19.422833,-99.174275,310,1
8,4e0788d1d1640223a49d9814,Hotel Lord,Doctores,Hotel,19.419112,-99.145252,341,1
9,4b0586fcf964a520577922e3,Hotel El Senador,Doctores,Hotel,19.418334,-99.145423,423,1


In [43]:
#Map Hotels and Hostels

colors = ['red','blue']

map_hotels = folium.Map(location=[latitude, longitude], zoom_start = 12.5)  #Create Map
for lat, lng, label,onehot in zip(hotels['Latitude'], hotels['Longitude'], hotels['Name'], hotels['One-hot Encoding']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat,lng],
        radius = 4,
        popup = label,
        color = colors[onehot],
        fill= True,
        fill_opacity = 0.7).add_to(map_hotels)
    
map_hotels = add_categorical_legend(m, 'My title',
                             colors = ['#000','#03cafc'],
                           labels = ['Heat', 'Cold'])

map_hotels

NameError: name 'add_categorical_legend' is not defined

There seems to be three clusters of hotels in Mexico City. The first one and largest one is located in the city centre. Note this cluster is composed of hotels and hostels. The second cluster is located right next to Mexico City's airport. The third cluster is located by a park (Bosque de Chapultepec) west o the city centre.

In [187]:
#Get all stations with gyms nearby. 
gyms = df_venues.loc[(df_venues['Category'] == 'Gym') | (df_venues['Category'] == 'Gym / Fitness Center')]
gyms = gyms.drop_duplicates(subset='Name').reset_index(drop=True)
gyms

,index,Venue Id,Name,Station,Category,Latitude,Longitude,Distance (m)
0,13,4cc47b0b06c25481ac0c9747,Sport Life,Aculco,Gym,19.373415,-99.111936,409
1,22,4f7c4235e4b02e009087f0e5,Gym & Fitness Central,Agrícola Oriental,Gym,19.406980,-99.070448,245
2,172,50f1b311e4b00ee580d1e016,Gym,Aragón,Gym,19.454440,-99.095443,359
3,264,4cea84cef1c6236a44236bf0,Aquatik Sport Center,Balbuena,Gym,19.421166,-99.099746,353
4,290,58f4c4c1610f040fd39305ba,Sports World,Barranca del Muerto,Gym / Fitness Center,19.361693,-99.191093,152
...,...,...,...,...,...,...,...,...
100,4158,52f9755711d2d0dc1361ef07,FitFlow Pole Dance and Fitness San Rafael,San Cosme,Gym / Fitness Center,19.440219,-99.163586,309
101,4184,4d56bc1e96ed5481e0441870,Pedro's gym,San Joaquín,Gym,19.446160,-99.187562,449
102,4502,4ceaf62ff865370492e1bfc4,Gym EHD Talisman,Talismán,Gym,19.473297,-99.107726,113
103,4527,4d267d11915fa0933b19eb09,CAFI EHD Tlalpan,Tasqueña,Gym / Fitness Center,19.341259,-99.143312,330


In [151]:
#Map Gyms and Fitness Centers

map_gyms = folium.Map(location=[latitude, longitude], zoom_start = 11)
for lat, lng, label in zip(gyms['Latitude'], gyms['Longitude'], gyms['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng], radius = 4, popup = label, fill= True, fill_opacity = 0.7).add_to(map_gyms)

map_gyms

# 5.  K-Means Clustinering 

In [152]:
low_venue_stations_list  = low_venue_stations['Station'].tolist()
low_venue_stations_list

[]

In [153]:
#Remove records from Stations that have 4 venues or less.
df_venues = df_venues[~df_venues['Station'].isin(low_venue_stations_list)].reset_index()

In [154]:
df_venues

,index,Venue Id,Name,Station,Category,Latitude,Longitude,Distance (m)
0,0,5ab96a96e1f228669fc9ea99,La Esperanza,Acatitla,Bakery,19.363365,-99.004722,196
1,1,56807381498eabd7d85a388f,little caesar,Acatitla,Pizza Place,19.364720,-99.006484,62
2,2,4d138c26d1848cfaede3c471,Toks,Acatitla,Mexican Restaurant,19.366074,-99.005900,148
3,3,51fd69e5498ec316901c73db,Tender's Coffe,Acatitla,Diner,19.366628,-99.004765,241
4,4,513cc414e4b00e248b4a7123,Domino's Pizza,Acatitla,Pizza Place,19.366939,-99.005129,257
...,...,...,...,...,...,...,...,...
4621,4621,4efdd4958231bcb701e18efe,Chilaquiles de la 178,Terminal Aérea,Mexican Restaurant,19.434192,-99.090906,343
4622,4622,4d9504cb2bd6f04d6f931550,7-Eleven,Terminal Aérea,Convenience Store,19.435741,-99.090765,394
4623,4623,4ed3aecff5b9832addf8fc4d,Naturally Fast,Terminal Aérea,Fast Food Restaurant,19.435142,-99.084192,397
4624,4624,4dc701cbfa76d685ce08401c,Educal,Terminal Aérea,Bookstore,19.435098,-99.084009,412


In [155]:
#Preapre data for KMeans 

#ONE HOT ENCODING
mexico_city_onehot = pd.get_dummies(df_venues[['Category']], prefix='', prefix_sep='')

#Add Station Column to ONEHOT dataframe
mexico_city_onehot.insert(0,'Station', df_venues['Station'])

mexico_city_onehot

,Station,ATM,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,American Restaurant,...,Veterinarian,Video Game Store,Warehouse Store,Watch Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Acatitla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Acatitla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Acatitla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Acatitla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Acatitla,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,Terminal Aérea,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4622,Terminal Aérea,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4623,Terminal Aérea,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4624,Terminal Aérea,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [156]:
mexico_city_grouped  = mexico_city_onehot.groupby('Station').mean().reset_index()
mexico_city_grouped

,Station,ATM,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,American Restaurant,...,Veterinarian,Video Game Store,Warehouse Store,Watch Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,Acatitla,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aculco,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Agrícola Oriental,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Allende,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Apatlaco,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,Talismán,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
139,Tasqueña,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
140,Tepalcates,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.090909,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
141,Tepito,0.0,0.000000,0.0,0.0,0.0,0.00000,0.00000,0.00000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [157]:
mexico_city_grouped_clustering = mexico_city_grouped.drop('Station',1)
mexico_city_grouped_clustering.head()

,ATM,Accessories Store,Advertising Agency,African Restaurant,Airport,Airport Gate,Airport Lounge,Airport Service,American Restaurant,Antique Shop,...,Veterinarian,Video Game Store,Warehouse Store,Watch Shop,Winery,Wings Joint,Women's Store,Yoga Studio,Zoo,Zoo Exhibit
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [158]:
from sklearn.cluster import KMeans

In [159]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(mexico_city_grouped_clustering)

In [160]:
kmeans.labels_[0:10]

array([0, 2, 2, 1, 2, 2, 2, 1, 1, 2])

In [161]:
cluster_labels = mexico_city_grouped[['Station']]
cluster_labels.insert(1, 'Cluster Labels', kmeans.labels_)
cluster_labels

,Station,Cluster Labels
0,Acatitla,0
1,Aculco,2
2,Agrícola Oriental,2
3,Allende,1
4,Apatlaco,2
...,...,...
138,Talismán,2
139,Tasqueña,1
140,Tepalcates,1
141,Tepito,1


In [162]:
#Add cluster labels to Stations dataframe.
merged_test = pd.merge(left=df_stations, right=cluster_labels, how = 'left', left_on='Station', right_on='Station')

In [163]:
#fill empty cells with int. 5. 
merged_test = merged_test.fillna(5)

In [164]:
merged_test = merged_test.astype({'Cluster Labels': 'int'})

In [165]:
merged_test.head()

,Station,Line,Year-Built,Grade,Latitude,Longitude,Closest Station,Distance to Closest Station,Cluster Labels
0,Acatitla,A,1991,At-grade,19.364737,-99.005892,Santa Marta,1272,0
1,Aculco,8,1994,At-grade,19.374069,-99.108095,Apatlaco,599,2
2,Agrícola Oriental,A,1991,At-grade,19.404835,-99.069901,Pantitlán,1188,2
3,Allende,2,1970,Underground,19.435514,-99.137406,Bellas Artes,484,1
4,Apatlaco,8,1994,At-grade,19.379292,-99.109597,Aculco,599,2


In [166]:
import numpy as np
import matplotlib.cm as cm 
import matplotlib.colors as colors

In [167]:
colors  = ['red','blue','green','cyan','black','magenta']
colors

['red', 'blue', 'green', 'cyan', 'black', 'magenta']

In [168]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(merged_test['Latitude'], merged_test['Longitude'], merged_test['Station'], merged_test['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=colors[cluster-1],
        fill=True,
        fill_color=colors[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)


In [169]:
map_clusters

In [152]:
len(kmeans.labels_)

163

In [153]:
df_venues.shape

(5251, 7)

In [154]:
test = pd.merge(left=df_venues, right=cluster_labels, how = 'left', left_on='Station', right_on='Station')
test

,index,Name,Category,Station,Latitude,Longitude,Distance (m),Cluster Labels
0,0,La Esperanza,Bakery,Acatitla,19.363365,-99.004722,196,1
1,1,Toks,Mexican Restaurant,Acatitla,19.366074,-99.005900,148,1
2,2,little caesar,Pizza Place,Acatitla,19.364720,-99.006484,62,1
3,3,Tender's Coffe,Diner,Acatitla,19.366628,-99.004765,241,1
4,4,Domino's Pizza,Pizza Place,Acatitla,19.366939,-99.005129,257,1
...,...,...,...,...,...,...,...,...
5246,5246,Café del Centro,Café,Zócalo,19.435825,-99.132837,286,1
5247,5247,Terraza La Deli,Italian Restaurant,Zócalo,19.435857,-99.131562,322,1
5248,5248,Café Ciudad de México,Café,Zócalo,19.432141,-99.134446,203,1
5249,5249,MineraliA Outlet,Arts & Crafts Store,Zócalo,19.430370,-99.135395,413,1


In [155]:
test.groupby('Cluster Labels').size()

Cluster Labels
0     517
1    3852
2     882
dtype: int64

In [156]:
cluster_0 = test.loc[test['Cluster Labels'] == 0]
cluster_0

,index,Name,Category,Station,Latitude,Longitude,Distance (m),Cluster Labels
509,509,LeFranm Spa,Spa,Calle 11,19.320913,-99.084540,135,0
510,510,La Esperanza,Bakery,Calle 11,19.320260,-99.083836,200,0
511,511,Pizzas Luigui,Pizza Place,Calle 11,19.320266,-99.087562,193,0
512,512,Toks,Mexican Restaurant,Calle 11,19.320555,-99.085270,49,0
513,513,tacos al carbon el moreno,Taco Place,Calle 11,19.319429,-99.087881,253,0
...,...,...,...,...,...,...,...,...
4659,4659,Barbacoa EMY,Taco Place,Tlaltenco,19.297955,-99.023958,398,0
4660,4660,Pancita Las 3 Lolas,Mexican Restaurant,Tlaltenco,19.295702,-99.021126,345,0
4661,4661,Metro Tlaltenco - L12,Metro Station,Tlaltenco,19.294310,-99.023802,32,0
4662,4662,Rincon Del Empresario,Soccer Stadium,Tlaltenco,19.293128,-99.022112,251,0


In [157]:
most_common_categories_cluster_0 = cluster_0.groupby('Category').size().sort_values(ascending=False).to_frame()
most_common_categories_cluster_0[0:10]

,0
Category,
Mexican Restaurant,136
Taco Place,45
Restaurant,21
Convenience Store,17
Bakery,17
Seafood Restaurant,17
Bar,14
Pizza Place,11
Fast Food Restaurant,10


In [158]:
cluster_1 = test.loc[test['Cluster Labels'] == 1]
cluster_1

,index,Name,Category,Station,Latitude,Longitude,Distance (m),Cluster Labels
0,0,La Esperanza,Bakery,Acatitla,19.363365,-99.004722,196,1
1,1,Toks,Mexican Restaurant,Acatitla,19.366074,-99.005900,148,1
2,2,little caesar,Pizza Place,Acatitla,19.364720,-99.006484,62,1
3,3,Tender's Coffe,Diner,Acatitla,19.366628,-99.004765,241,1
4,4,Domino's Pizza,Pizza Place,Acatitla,19.366939,-99.005129,257,1
...,...,...,...,...,...,...,...,...
5246,5246,Café del Centro,Café,Zócalo,19.435825,-99.132837,286,1
5247,5247,Terraza La Deli,Italian Restaurant,Zócalo,19.435857,-99.131562,322,1
5248,5248,Café Ciudad de México,Café,Zócalo,19.432141,-99.134446,203,1
5249,5249,MineraliA Outlet,Arts & Crafts Store,Zócalo,19.430370,-99.135395,413,1


In [159]:
most_common_categories_cluster_1 = cluster_1.groupby('Category').size().sort_values(ascending=False).to_frame()
most_common_categories_cluster_1[0:10]

,0
Category,
Mexican Restaurant,300
Taco Place,186
Coffee Shop,183
Restaurant,124
Ice Cream Shop,115
Bakery,110
Pizza Place,98
Café,92
Seafood Restaurant,71


In [160]:
cluster_2 = test.loc[test['Cluster Labels'] == 2]
cluster_2

,index,Name,Category,Station,Latitude,Longitude,Distance (m),Cluster Labels
23,23,El Camioncito,Burrito Place,Agrícola Oriental,19.407263,-99.069180,280,2
24,24,"Tacos ""el Güero""",Taco Place,Agrícola Oriental,19.406273,-99.067270,319,2
25,25,Gym & Fitness Central,Gym,Agrícola Oriental,19.406980,-99.070448,245,2
26,26,El Fogoncito,Taco Place,Agrícola Oriental,19.405402,-99.069214,95,2
27,27,Mercado Express Soriana,Convenience Store,Agrícola Oriental,19.404357,-99.068011,205,2
...,...,...,...,...,...,...,...,...
5134,5134,Mercado Zapotitlan,Farmers Market,Zapotitlán,19.295315,-99.036830,272,2
5135,5135,Terraza VIP,Bar,Zapotitlán,19.298323,-99.036516,271,2
5136,5136,Mercado La Conchita,Market,Zapotitlán,19.294137,-99.035610,295,2
5137,5137,Olympus Gym,Gym,Zapotitlán,19.296296,-99.037569,309,2


In [161]:
most_common_categories_cluster_2 = cluster_2.groupby('Category').size().sort_values(ascending=False).to_frame()
most_common_categories_cluster_2[0:10]

,0
Category,
Taco Place,177
Mexican Restaurant,110
Bakery,36
Convenience Store,27
Gym,22
Burger Joint,19
Bar,18
Coffee Shop,17
Seafood Restaurant,17
